<a href="https://colab.research.google.com/github/ErikFantomex/MPI_Curso/blob/master/Lecturas/LecturaComunicadoresygrupos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grupos
Esta sección describe las funciones que permiten la manipulación de grupos de procesos en MPI. Estas operaciones son locales y su ejecución no requiere de comunicación interprocesos. MPI permite la manipulación de grupos fuera de un comunicador pero solo pueden ser usados para paso de mensajes como parte de un comunicador. 

## MPI_GROUP_SIZE
Regresa el número de procesos en el grupo. Si group=MPI\_GROUP\_EMPTY, entonces size=0.  El valor de retorno puede ser 

1. MPI\_SUCCESS Significa que no hubo error. La función se completó exitosamente. 
2. MPI\_ERR\_GROUP Significa que se pasó como argumento un grupo nulo.
3. MPI\_ERR\_ARG     Significa que hay un argumento no válido.


## MPI_GROUP_RANK


Esta función regresa el rango correspondiente al proceso que hizo la llamada a la función, dentro del grupo $group$. Si el proceso no es miembro de $group$ entonces $rank$ toma el valor MPI\_UNDEFINED. El valor del rango es un entero en $[0, size-1]$, donde $size$ es el valor regresado por MPI\_Group\_size().

## MPI_COMM_GROUP


##C

In [ ]:
%%writefile grupo_world.c
// Ejemplo con el grupo default 
#include "mpi.h"
#include <stdio.h>
int main(  ){
   int myID_grupo, numProcs_grupo;	
   MPI_Group grupo1;  	

   MPI_Init(NULL,NULL);

   // Obtiene un manejador al grupo del comunicador por default 
   MPI_Comm_group(MPI_COMM_WORLD, &grupo1);
   	
   MPI_Group_size(grupo1, &numProcs_grupo);   	
   MPI_Group_rank(grupo1, &myID_grupo);   
   printf("Somos %d procesos en el grupo, mi ID es %d \n",numProcs_grupo,myID_grupo);
   	
   MPI_Finalize();
   return 0;
}

Writing grupo_world.c


In [ ]:
%%script bash
mpicc grupo_world.c -o grupo_world
mpirun --allow-run-as-root -np 6 grupo_world

Somos 6 procesos en el grupo, mi ID es 2 
Somos 6 procesos en el grupo, mi ID es 5 
Somos 6 procesos en el grupo, mi ID es 3 
Somos 6 procesos en el grupo, mi ID es 1 
Somos 6 procesos en el grupo, mi ID es 4 
Somos 6 procesos en el grupo, mi ID es 0 


##C++

In [ ]:
%%writefile grupo_vacio.c
// Ejemplo con un grupo vacio 
#include "mpi.h"
#include <stdio.h>
int main(  ){
   MPI_Init(NULL,NULL);   	
   int myID,numProcs;	   
   MPI_Group grupo1 = MPI_GROUP_EMPTY;
   
   MPI_Group_size(grupo1, &numProcs);
   MPI_Group_rank(grupo1, &myID);   
   printf("Somos %d procesos, mi ID es %d \n",numProcs,myID);  
    	
   MPI_Finalize();
   return 0;
}

Writing grupo_vacio.c


In [ ]:
%%script bash
mpicc grupo_vacio.c -o grupo_vacio
mpirun --allow-run-as-root -np 5 grupo_vacio

Somos 0 procesos, mi ID es -32766 
Somos 0 procesos, mi ID es -32766 
Somos 0 procesos, mi ID es -32766 
Somos 0 procesos, mi ID es -32766 
Somos 0 procesos, mi ID es -32766 


## MPI_GROUP_COMPARE
Compara la relación entre dos grupos de procesos dados como argumentos,  *group1* y *group*.  El valor en *result* puede ser alguno de los siguientes

1. MPI\_IDENT si los procesos en los dos grupos son idénticos y tienen rangos iguales, 
2. MPI\_SIMILAR si los procesos son idénticos pero sus rango son diferentes, 
3. MPI\_UNEQUAL para cualquier otra relación entre los grupos.

La función regresa MPI\_SUCCESS si se llevó a cabo de forma correcta, en caso contrario regresa un valor distinto (MPI\_ERR\_GROUP o MPI\_ERR\_ARG), para indicar que alguno de los grupos pasados como argumento son nulos o no válidos, o bien que alguno de los argumentos son no válidos o que no se pudo detectar el tipo de error.

## Constructores
Los constructores de grupo son usados para construir grupos nuevos a partir de grupos existentes usando un conjunto de operaciones. Las operaciones que podemos usar para crear nuevos grupos a partir de uno o dos grupos dados son


* inclusión, 
* exclusión,
* unión,
* diferencia
* intersección

Nótese que las operaciones MPI\_Group\_union(), MPI\_Group\_intersection() y MPI\_Group\_difference() el orden de los procesos en el grupo de salida primero está determinado  por el orden del $grupo1$ y después, si es necesario, por el orden en $group2$. La unión y la intersección no son conmutativos pero son asociativas. El nuevo grupo resultante puede estar vacío en cuyo caso será igual a MPI\_GROUP\_EMPTY. 

### MPI_GROUP_INCL

Se crea un nuevo grupo, llamado $newgroup$, a partir de la inclusión de los $n$ miembros del grupo $group$, especificados en el arreglo $ranks$. El arreglo $ranks$ debe contener rangos válidos de elementos de $group$, sin repetir alguno, en otro caso será una llamada errónea. Si $n=0$, entonces $newgroup$=MPI\_GROUP\_EMPTY. 



In [ ]:
%%writefile grupos_incl.c
/*
 * Crea un nuevo grupo llamado <newGroup> a partir del grupo <worldGroup>
 * Se incluye en el nuevo grupo a un conjunto de procesos especificados en
 * el arreglo <ranks>.
 * Despues se comparan ambos grupos y se manda un mensaje con el resultado 
 * de la comparacion (iguales, semejantes o diferentes)
*/
#include "mpi.h"
#include <stdio.h>
#include <stdlib.h>
int main(  ){
  MPI_Init( NULL , NULL );        
  int i, myId ,myId_new, res;
  int numprocs, nElems, *ranks;
  MPI_Group worldGroup, newGroup;

   MPI_Comm_group(MPI_COMM_WORLD,&worldGroup);
   MPI_Group_size (worldGroup,&numprocs);	        
   MPI_Group_rank (worldGroup,&myId);		
		
   nElems   = ( numprocs+1)/2; // numero de procesos pares
   ranks = (int*)malloc(nElems*sizeof(int)); if(ranks==NULL){ printf("Memoria insuficiente \n"); MPI_Finalize(); return 0; }

   for(i=0;i<nElems;i++) ranks[i]=2*i;

   MPI_Group_incl(worldGroup,nElems,ranks,&newGroup);
   MPI_Group_rank(newGroup,&myId_new);

   if(myId_new!=MPI_UNDEFINED)
      printf("\n [ID en WORLD_GROUP %d][ID en el newGroup %d]\n",myId,myId_new);
		MPI_Barrier(MPI_COMM_WORLD);
   // comparando al worldGroup y al newGroup
   MPI_Group_compare(worldGroup,newGroup,&res);
   if (myId==0)
      if(res==MPI_IDENT )printf("\n *** Grupos iguales\n");
      else
         if(res==MPI_SIMILAR)printf("\n *** Grupos similares\n");
         else  // res==MPI_UNEQUAL 
            printf("\n *** Grupos diferentes\n");

   if(myId_new!=MPI_UNDEFINED) MPI_Group_free(&newGroup);
        
   MPI_Finalize();
   return 0;
}

Writing grupos_incl.c


In [ ]:
%%script bash
mpicc grupos_incl.c -o ej_incl
mpirun --allow-run-as-root -np 9 ej_incl



 [ID en WORLD_GROUP 8][ID en el newGroup 4]

 [ID en WORLD_GROUP 4][ID en el newGroup 2]
 [ID en WORLD_GROUP 0][ID en el newGroup 0]

 [ID en WORLD_GROUP 6][ID en el newGroup 3]

 [ID en WORLD_GROUP 2][ID en el newGroup 1]

 *** Grupos diferentes


### MPI_GROUP_EXCL
Permite crear un nuevo grupo, llamado $newgroup$, consistente de todos los procesos de $group$ que no hayan sido excluidos. Los $n$ procesos a excluirse están especificados en el arreglo $ranks$. El orden de los procesos en el nuevo grupo es idéntico al que tenían en $group$. El arreglo $ranks$ debe contener rangos válidos de elementos de $group$, sin repetir alguno, en otro caso será una llamada errónea. Si $n=0$, entonces $newgroup$ será idéntico a $group$.


In [ ]:
%%writefile grupos_excl.c

/* Crea un nuevo grupo llamado <newGroup> a partir del grupo <worldGroup>
 * excluyendo del nuevo grupo a un conjunto de procesos especificados en
 * el arreglo <ranks>
 * Despues se comparan ambos grupos y se manda un mensaje con el resultado de la 
 * comparacion (iguales, semejantes o diferentes)
*/

#include "mpi.h"
#include <stdlib.h>
#include <stdio.h>
int main(  ){
	int		i,res,world_Id,rank,numprocs,npp;
	int 	*procPares;	
	MPI_Group 	worldGroup, gImpares;

	MPI_Init(NULL,NULL);
	
	MPI_Comm_group(MPI_COMM_WORLD,&worldGroup);
	MPI_Group_size (worldGroup,&numprocs);	
	MPI_Group_rank (worldGroup,&world_Id);

	// formando el conjunto de los pares
  npp   = (numprocs+1)/2; // numero de procesos pares
  procPares = (int*)malloc(sizeof(int)*npp);
  for(i=0;i<npp;i++) 
		procPares[i] = 2*i;
		
	// formando un nuevo grupo apartir de <world_group>
	// eliminando los elementos en <procPares>
  	MPI_Group_excl(worldGroup, npp, procPares, &gImpares);
  	MPI_Group_rank(gImpares,&rank);    
	
	if(rank!=MPI_UNDEFINED) 
		printf("\n [ID en world_group %d][ID en gImpares %d]\n",world_Id,rank);
    
  // comparando al worldGroup y al gImpares
	MPI_Group_compare(worldGroup,gImpares,&res);
	if (world_Id==0)
      if(res==MPI_IDENT )printf("\n *** Grupos iguales\n");
      else
         if(res==MPI_SIMILAR)printf("\n *** Grupos similares\n");
         else  // res==MPI_UNEQUAL 
            printf("\n *** Grupos diferentes\n");
    
  	if(gImpares!=MPI_GROUP_NULL) MPI_Group_free(&gImpares);
    
	MPI_Finalize();
  	return 0;
}

Writing grupos_excl.c


In [ ]:
%%script bash
mpicc grupos_excl.c -o ej_excl
mpirun --allow-run-as-root -np 6 ej_excl


 [ID en world_group 5][ID en gImpares 2]

 [ID en world_group 1][ID en gImpares 0]

 [ID en world_group 3][ID en gImpares 1]

 *** Grupos diferentes


### MPI_GROUP_UNION
Un nuevo grupo $newgroup$ es creado a partir de la unión de dos grupos ya definidos, $group1$  y $group2$. Los miembros del primer grupo conservan su rango mientras que los miembros del segundo grupo tendrán rango nuevo en $newgroup$.


In [ ]:
%%writefile grupos_union.c

/* Crea un nuevo grupo llamado <newGroup> a partir de los grupos 
 * <gPares> y <gImpares> formados previamente
 *
 * Despues se comparan los grupos newGroup y worldGroup,  y se manda un mensaje con el resultado 
 * de la comparacion (iguales, semejantes o diferentes). 
 * Observese que newGroup contiene el mismo conjunto de procesos que 
 * <worldGroup> aunque estan etiquetados de manera diferente, por esta
 * razon son grupos "semejantes".
*/

#include "mpi.h"
#include <stdio.h>
#include <stdlib.h>

int main(  ){
   int i,myId,myid_grupoPares,rank,res;
   int numprocs,npares,*pares;
   int nimpares,*impares;
   MPI_Group worldGroup,gPares,gImpares,newGroup;

   MPI_Init( NULL , NULL );
	
   MPI_Comm_group(MPI_COMM_WORLD,&worldGroup);
   MPI_Group_size (worldGroup,&numprocs);	
   MPI_Group_rank (worldGroup,&myId);
     
   // formando un grupo que incluya solo a los pares de <worldGroup>
   npares   =(numprocs+1)/2;
   pares=(int*)malloc(sizeof(int)*npares);
   for(i=0;i<npares;i++)	pares[i]=2*i;
   MPI_Group_incl(worldGroup, npares,pares,&gPares);
	
   // formando un grupo que incluya solo a los impares de <worldGroup>
   nimpares=numprocs-npares;
   impares=(int*)malloc(sizeof(int)*nimpares);
   for(i=0;i<nimpares;i++) impares[i]=2*i+1;
   MPI_Group_incl(worldGroup, nimpares,impares,&gImpares);	
	
   // formando un nuevo grupo a partir de la union del grupo de los 
   // pares y el de los impares
   MPI_Group_union(gPares, gImpares, &newGroup);
   MPI_Group_rank(newGroup,&rank);
    
   if(rank!=MPI_UNDEFINED)
      printf("\n [ID en worldGroup= %d][ID en el grupo union= %d]\n",myId,rank);
		
   // comparando al worldGroup y al newgroup
   MPI_Group_compare(worldGroup,newGroup,&res);
   if (myId==0)
      if(res==MPI_IDENT )printf("\n *** Grupos iguales\n");
      else
         if(res==MPI_SIMILAR)printf("\n *** Grupos similares\n");
         else  // res==MPI_UNEQUAL 
            printf("\n *** Grupos diferentes\n");           
   MPI_Finalize();
   return 0;
}	

Writing grupos_union.c


In [ ]:
%%script bash
mpicc grupos_union.c -o ej_union
mpirun --allow-run-as-root -np 5 ej_union


 [ID en worldGroup= 0][ID en el grupo union= 0]


 [ID en worldGroup= 2][ID en el grupo union= 1]
 *** Grupos similares

 [ID en worldGroup= 3][ID en el grupo union= 4]


 [ID en worldGroup= 1][ID en el grupo union= 3]
 [ID en worldGroup= 4][ID en el grupo union= 2]


### MIP_GROUP_DIFFERENCE

El nuevo grupo consiste de los elementos que pertenecen a $group1$ que no pertenezcan a $group2$, tendrán el orden especificado en $group1$.

In [ ]:
%%writefile grupos_difference.c

/*
 * Crea un nuevo grupo llamado <gPares> a partir del grupo <worldGroup>
 * incluyendo del nuevo grupo a un conjunto de procesos especificados en
 * el arreglo <ranks>
 * Despues se crea un nuevo grupo llamado <newGroup> a partir de la 
 * diferencia entre los procesos en <worldGroup> y en <gPares>
*/
#include "mpi.h"
#include <stdio.h>
#include <stdlib.h>
int main(  ){
  int i,myId,myIdNew,res,myIdPar;
  int numprocs,nElems,*ranks;
  MPI_Group worldGroup,newGroup,gPares;

  MPI_Init( NULL , NULL );        
  MPI_Comm_group(MPI_COMM_WORLD,&worldGroup);
  MPI_Group_size (worldGroup,&numprocs);        
  MPI_Group_rank (worldGroup,&myId);		
  
  // formando un grupo incluyendo los elementos pares de <worldGroup>
  nElems   = ( numprocs+1)/2;
  ranks=(int*)malloc(nElems*sizeof(int)); if(ranks==NULL){printf("Memoria insuficiente \n"); MPI_Finalize();  return 0; }

  for(i=0;i<nElems;i++) ranks[i]=2*i;		

  MPI_Group_incl(worldGroup,nElems,ranks,&gPares);        
  MPI_Group_rank(gPares,&myIdPar);
  
  // formando un nuevo grupo a partir de la diferencia entre 
  // <worldGroup> y en <gPares>
  MPI_Group_difference(worldGroup,gPares,&newGroup);
  MPI_Group_rank(newGroup,&myIdNew);
  
  if(myIdNew!=MPI_UNDEFINED)
    printf("\n [ID en WORLD_GROUP %d][ID en el newGroup %d]\n",myId,myIdNew);		

  if(myIdPar!=MPI_UNDEFINED) MPI_Group_free(&gPares);
  if(myIdNew!=MPI_UNDEFINED) MPI_Group_free(&newGroup);
      
  MPI_Finalize();
  return 0;
}

Writing grupos_difference.c


In [ ]:
%%script bash
mpicc grupos_difference.c -o ej_diff
mpirun --allow-run-as-root -np 5 ej_diff


 [ID en WORLD_GROUP 3][ID en el newGroup 1]

 [ID en WORLD_GROUP 1][ID en el newGroup 0]


### MPI_GROUP_INTERSECTION
El nuevo grupo consiste de los elementos en el $group1$ que también están en $group2$, tendrán rangos en el mismo orden que en el $group1$.


In [ ]:
%%writefile grupos_intersection.c

/*
 * Crea un nuevo grupo llamado <gPares> a partir del grupo <worldGroup>
 * incluyendo del nuevo grupo a un conjunto de procesos especificados en
 * el arreglo <ranks>
 * Despues se crea un nuevo grupo llamado <newGroup> a partir de la 
 * interseccion entre los procesos en <worldGroup> y en <gPares>
*/
#include "mpi.h"
#include <stdio.h>
#include <stdlib.h>
int main(  ){
   int	i,myId,myIdNew,res,myIdPar;
   int numprocs,nElems,*ranks;
   MPI_Group worldGroup,newGroup,gPares;

   MPI_Init( NULL , NULL );        
   MPI_Comm_group(MPI_COMM_WORLD,&worldGroup);
   MPI_Group_size (worldGroup,&numprocs);        
   MPI_Group_rank (worldGroup,&myId);		
		
   // formando un grupo incluyendo los elementos pares de <worldGroup>
   nElems   = ( numprocs+1)/2;
   ranks=(int*)malloc(nElems*sizeof(int));
   if(ranks==NULL){printf("Memoria insuficiente \n"); MPI_Finalize();  return 0;}

   for(i=0;i<nElems;i++) ranks[i]=2*i;		
   MPI_Group_incl(worldGroup,nElems,ranks,&gPares);        
   
   // formando un nuevo grupo a partir de la interseccion entre 
   // <worldGroup> y en <gPares>
   MPI_Group_intersection(worldGroup,gPares,&newGroup);
   MPI_Group_rank(newGroup,&myIdNew);
   
   if(newGroup==MPI_GROUP_EMPTY ) 
   	printf("\nLa interseccion resultante es un grupo vacio\n\n");
   else
      if(myIdNew!=MPI_UNDEFINED)
         printf("\n [ID en WORLD_GROUP %d][ID en el newGroup %d]\n",myId,myIdNew);		

   if(myIdPar!=MPI_UNDEFINED) MPI_Group_free(&gPares);
   if(myIdNew!=MPI_UNDEFINED) MPI_Group_free(&newGroup);
        
   MPI_Finalize();
   return 0;
}

Writing grupos_intersection.c


In [ ]:
%%script bash
mpicc grupos_intersection.c -o ej_inters
mpirun --allow-run-as-root -np 5 ej_inters



 [ID en WORLD_GROUP 2][ID en el newGroup 1]

 [ID en WORLD_GROUP 4][ID en el newGroup 2]
 [ID en WORLD_GROUP 0][ID en el newGroup 0]


## MPI_GROUP_FREE(&grupo)



Destruye el grupo $group$. Una vez realizada la llamada $group$=MPI\_GROUP\_NULL. Nótese que liberar a un grupo, no libera al comunicador al cual pertenece. Para ello hay que usar la función MPI\_Comm\_free().

#Comunicadores

##MPI_COM_RANK
En la variable $rank$ regresa el identificador del proceso (un valor entero entre $0$ y $num\_procesos -1$) bajo el comunicador $comm$. Esta valor es relativo al grupo asociado con $comm$. Si la llamada es realizada como MPI\_Comm\_rank(MPI\_COMM\_WORLD,\&rank), regresa un valor en el contexto del comunicador global; si es utilizada la llamada MPI\_Comm\_rank(MPI\_COMM\_SELF,\&rank) entonces $rank=0$. Devolverá un valor distinto de MPI\_SUCCESS si el comunicador no es válido.

##MPI_COMM_SIZE
Si $comm$ es un intracomunicador, entonces regresa el tamaño del grupo asociado con $comm$. La función regresará un valor distinto de MPI\_SUCCESS si el comunicador no es válido. 

##Constructores
En MPI hay varias formas de crear un nuevo comunicador. Estas formas son


* duplicar un intracomunicador existente,
* dividir un intracomunicador en dos o más sub-intracomunicadores,
* modificar un grupo de procesos en un intracomunicador existente, para crear un nuevo comunicador en base al grupo modificado.


### MPI_COMM_DUP
Duplica un comunicador existente. Es decir, crea un nuevo intracomunicador con los mismo atributos que $comm$. El nuevo comunicador tendrá el mismo grupo asociado que $comm$ pero con un contexto distinto.




### MPI_COMM_CREATE
Genera un comunicador con $group$ como grupo asociado. Ningún atributo se propaga de $comm$ a $newcomm$. Regresa MPI\_COMM\_NULL para procesos que no están en el grupo. $group$ debe ser un subconjunto del grupo asociado al comunicador $comm$.

In [ ]:
%%writefile comunicadores.c
/* Crea un nuevo grupo llamado <newGroup> a partir del grupo <worldGroup>
 * incluyendo del nuevo grupo a un conjunto de procesos especificados en
 * el arreglo <ranks>
 * Despues crea un comunicador llamado <newComm> con el grupo <newGroup>,
 * obtiene el Id que corresponde en el nuevo comunicador y 
 * lo manda a imprimir
*/
#include "mpi.h"
#include <stdio.h>
#include <stdlib.h>
int main(  ){
   int i,myId,myIdComm,myIdGroup;
   int numprocs,nElems,*ranks;
   MPI_Group worldGroup,newGroup;
   MPI_Comm newComm;

   MPI_Init( NULL , NULL );        
   MPI_Comm_group(MPI_COMM_WORLD,&worldGroup);
   MPI_Group_size (worldGroup,&numprocs);	        
   MPI_Group_rank (worldGroup,&myId);		
		
   nElems=(numprocs+1)/2;
   ranks=(int*)malloc(nElems*sizeof(int));
   if(ranks==NULL){
      printf("Memoria insuficiente \n");
      MPI_Finalize();
      return 0;
   }
   for(i=0;i<nElems;i++) ranks[i]=2*i;
   MPI_Group_incl(worldGroup,nElems,ranks,&newGroup);  
   MPI_Group_rank(newGroup,&myIdGroup);   
   
   MPI_Comm_create(MPI_COMM_WORLD,newGroup,&newComm);   
   if(newComm!=MPI_COMM_NULL){
      MPI_Comm_rank(newComm,&myIdComm);      
      printf("\n [ID en COMM_WORLD %d][ID en el newComm %d]\n",myId,myIdComm);
   }	
   if(myIdGroup!=MPI_UNDEFINED) MPI_Group_free(&newGroup);
   if(newComm!=MPI_COMM_NULL) MPI_Comm_free(&newComm);
   
   MPI_Finalize();
   return 0;
}

Writing comunicadores.c


In [ ]:
%%script bash
mpicc comunicadores.c -o ej_comm
mpirun --allow-run-as-root -np 5 ej_comm



 [ID en COMM_WORLD 0][ID en el newComm 0]

 [ID en COMM_WORLD 4][ID en el newComm 2]

 [ID en COMM_WORLD 2][ID en el newComm 1]


### MPI_COMM_SPLIT
Esta función divide un intracomunicador existente en dos o mas sub-itercomunicadores. La función parte el grupo asociado con $comm$ en subgrupos disjuntos, uno para cada valor de $color$. Cada subgrupo contiene todos los procesos del mismo valor en $color$. Dentro de cada grupo, los procesos son ordenados de acuerdo al valor del argumento $key$. Un nuevo dominio de comunicación es creado y se regresa un manejador en $newcomm$. Un proceso puede contener el valor MPI\_UNDEFINED en $color$ y no formará parte del nuevo comunicador en cuyo caso, $newcomm=$MPI\_COMM\_NULL. Esta es una llamada colectiva pero cada proceso puede proporcionar un valor distinto en $color$ y $key$. Los valores en color deben ser no-negativos.

In [ ]:
%%writefile comunicadores_split.c
/* *****
 * Crea un nuevo comunicador <newComm> a partir de los valores 
 * <color> y <key>
 * <newComm> en realidad representa <color> comunicadores
 * con los procesos ordenados segun <key>  
 * Despues se obtiene el valor del identificador en el comunicador
 * correspondiente a cada proceso y se manda a imprimir junto a <myId>
 **** */
#include "mpi.h"
#include <stdio.h>
int main( ){
   int myId,np,myIdComm,npComm,color,key;
   MPI_Comm	newComm;

   MPI_Init( NULL,NULL );
   MPI_Comm_size (MPI_COMM_WORLD,&np);
   MPI_Comm_rank (MPI_COMM_WORLD,&myId);

   if(np<2){
      if(myId==0) printf("Deben utilizarse al menos %d procesos en la ejecucion\n",2);
      MPI_Finalize();
      return 0;
   }

   color= myId%2; // se crean 2 comunicadores
   key  = -myId; // se ordenaran en orden inverso al actual
   
   MPI_Comm_split(MPI_COMM_WORLD,color,key,&newComm);
   if(newComm != MPI_COMM_NULL){
      MPI_Comm_rank (newComm,&myIdComm);
      MPI_Comm_size (newComm,&npComm);
      if (myIdComm==0) 
      	printf("Color %d, Numero de procesos en el nuevo comunicador =%d\n",color,npComm);
         
      printf("\n[Color %d] [myid nuevo, myID viejo]=[%d,%d] \n",color,myIdComm,myId);
   }
	
   	if(newComm!=MPI_COMM_NULL) MPI_Comm_free(&newComm);

   MPI_Finalize();
   return 0;
}

Writing comunicadores_split.c


In [ ]:
%%script bash
mpicc comunicadores_split.c -o ej_split
mpirun --allow-run-as-root -np 5 ej_split


[Color 0] [myid nuevo, myID viejo]=[2,0] 

Color 1, Numero de procesos en el nuevo comunicador =2

[Color 1] [myid nuevo, myID viejo]=[0,3] 

[Color 0] [myid nuevo, myID viejo]=[1,2] 
Color 0, Numero de procesos en el nuevo comunicador =3

[Color 0] [myid nuevo, myID viejo]=[0,4] 
[Color 1] [myid nuevo, myID viejo]=[1,1] 


## Destructor



Destruye el comunicador $comm$, después de esto $comm=$MPI\_COMM\_NULL. Cualquier operación pendiente en este comunicador se completará normalmente.

Es un error intentar liberar MPI\_COMM\_NULL.